# DSCI-100 Project

In [10]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(tidymodels)


Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ modeldata    1.4.0     ✔ workflows    1.1.4
✔ parsnip      1.2.1     ✔ workflowsets 1.0.1
✔ recipes      1.1.0     ✔ yardstick    1.3.1

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/



## Introduction
This project will be analyzing data obtained from [a research group in Computer Science at UBC](https://plai.cs.ubc.ca/), led by [Frank Wood](https://www.cs.ubc.ca/~fwood/). Specifically, these data are obtained from numerous players' actions in the video game Minecraft. In interest of collecting data efficiently, this analysis will focus on which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

### File players.csv:
- `experience` Refers to the player's experience with Minecraft as the following:
    * Pro
    * Veteran
    * Regular
    * Amateur
    * Beginner
- `subscribe` Refers to whether the player has subscribed to game-related newsletters.
- `hashedEmail` Player email, obfuscated for privacy.
- `played_hours` Player's total hours spent playing.
- `name` Player's name.
- `gender` Player's gender.
- `age` Player's age.

### File sessions.csv:
- `hashedEmail` Player email, obfuscated for privacy.
- `start_time` Session start time in human readable time.
- `end_time` Session end time in human readable time.
- `original_start_time` Session start time in Epoch time.
- `original_end_time` Session end time in Epoch time.

### The Big Question
Can we predict a player's total hours based on their experience and age? <br>
In this context, session data becomes redundant as they are obtained after recuiting the player. If we want to recruit players before they actually start playing on the server, we can't use session data in our prediction.